# Project 2
Keara Hayes, Atticus Chong, Zachary Malzahn, Evelyn Fuhrman

In [1]:
import numpy as np
from numpy.linalg import norm
import math
import matplotlib.pyplot as plt
import astropy.constants as _ac
import astropy.units as _au
from numpy import pi

In [ ]:
###importing constants

# solar mass, radius, luminosity
Msun = _ac.M_sun.value
Rsun = _ac.R_sun.value
Lsun = _ac.L_sun.value

# physical constants from astropy, all in MKS units
G = _ac.G.value
h = _ac.h.value
hbar = _ac.hbar.value
m_e = _ac.m_e.value
m_p = _ac.m_p.value
m_n = _ac.m_n.value
m_u = _ac.u.value
c = _ac.c.value
kB = _ac.k_B.value
pc = _ac.pc.value
au = _ac.au.value
year = _au.year.to(_au.second)
sigmaSB = _ac.sigma_sb.value

In [ ]:
def density(p, mue):
    """
    Arguments
        p
            electron degeneracy pressure (Pascal)
        mue
            baryon/electron ratio  
        mu
            amu
        h
            Planck's constant
        me
            electron mass
        
    Returns
        mass density (kg/m**3)
    """
    
    # replace following lines with body of routine
    rho = ((p*((2/5)*((3/8*pi)**2/3)*(h**2/(2*me)))**-1)**(3/5))*mu*mue
    return rho

In [ ]:
def central_values(Pc,delta_m,mue):
    """
    Constructs the boundary conditions at the edge of a small, constant density 
    core of mass delta_m with central pressure P_c
    
    Arguments
        Pc
            central pressure (units = ?)
        delta_m
            core mass (units = ?)
        mue
            nucleon/electron ratio
    
    Returns
        z = array([ r, p ])
            central values of radius and pressure (units = ?)
    """
    z = np.zeros(2)
    
    z[0] = density(Pc, mue)
    z[1] = Pc   # this feels weird but like...the phrasing suggests to me this is what it wants???
    # compute initial values of z = [ r, p ]
    return z

## Old Project 1 Shite

# 1)

In [ ]:
def rk4(f,t,z,h,args=()):
    """    
    Arguments
        f(t,z,...)
            function that contains the RHS of the equation dz/dt = f(t,z,...)
    
        t (scalar)
            current time
            
        z (array-like)
            function value
            
        h (scalar)
            step size
    
        args (tuple, optional)
            additional arguments to pass to f
    
    Returns
        znew = z(t+h)
    """
   
    if not isinstance(args,tuple):
        args = (args,)

    k1 = f(t, z, *args)

    k2 = f(t + h/2, z + (h/2)*k1, *args)

    k3 = f(t + h/2, z + (h/2)*k2, *args)

    k4 = f(t + h, z + h*k3, *args)
    
    return z + (h/6)*(k1 + 2*k2 + 2*k3 + k4)

In [ ]:
def integrate_orbit(z0,m,tend,h,method='RK4'):
    """
    Integrates orbit starting from an initial position and velocity from t = 0 
    to t = tend.
    
    Arguments:
        z0 (scalar)
            initial function value
        
        m (scalar)
            mass of star in units of M_sun
    
        tend (scalar)
            final timestep
    
        h (scalar)
            incrementing stepsize
    
        method ('Euler', 'RK2', or 'RK4')
            identifies which stepper routine to use (default: 'RK4')
    Returns
        ts, Xs, Ys, KEs, PEs, TEs := arrays of time, x postions, y positions, 
        and energies (kin., pot., total) 
    """

    # set the initial time and phase space array
    t = 0.0
    z = z0

    # expected number of steps
    Nsteps = int(tend/h)+1

    # arrays holding t, x, y, kinetic energy, potential energy, and total energy
    ts = np.zeros(Nsteps) # time
    Xs = np.zeros(Nsteps) # x-axis position
    Ys = np.zeros(Nsteps) # y-axis position
    KEs = np.zeros(Nsteps) # kinetic energy
    PEs = np.zeros(Nsteps) # potential energy
    TEs = np.zeros(Nsteps) # total energy

    # store the initial point
    ts[0] = t
    Xs[0] = z[0]
    Ys[0] = z[1]
    KEs[0]=(1/2)*m*np.linalg.norm(z[2:4])**2
    PEs[0]=-m/np.linalg.norm(z[0:2])
    TEs[0]=KEs[0]+PEs[0]

    # select the stepping method
    advance_one_step = integration_methods[method]
    
    # run through the steps
    for step in range(1,Nsteps):
        z = advance_one_step(derivs,t,z,h,args=m)  # new z
        t += h                                     # step forward in time
        
        # store values
        ts[step] = t
        dzdt = derivs(t,z,m)
        v = dzdt[0:2]  # define velocity at this step
        acc = dzdt[2:4] # define acceleration at this step
        
        # fill in with assignments for Xs, Ys, KEs, PEs, TEs
        Xs[step] = z[0]
        Ys[step] = z[1]
        KEs[step] = kinetic_energy(v)
        PEs[step] = potential_energy(z[0:2], m)
        TEs[step] = total_energy(z,m)
    
    return ts, Xs, Ys, KEs, PEs, TEs
    
def set_initial_conditions(a, m, e):
    """
    set the initial conditions for the orbit.  The orientation of the orbit is 
    chosen so that y0 = 0.0 and vx0 = 0.0.
    
    Arguments
        a (scalar)
            semi-major axis in AU
        m (scalar)
            total mass in Msun
        e (scalar)
            eccentricity ( x0 = (1+e)*a )
    
    Returns:
    [x0, y0, vx0, vy0], eps0, Tperiod := initial position and velocity, energy, 
        and period
    """
    eps0=-(m/(2*a))
    T=(np.pi/math.sqrt(2))*m*(np.abs(eps0)**(-3/2))
    x0=(1+e)*a
    y0=0
    vx0=0
    vy0=math.sqrt(2*eps0+2*(m/x0))
    r_init=np.array([x0,y0])
    v_init=np.array([vx0,vy0])
    z_init=np.append(r_init,v_init)
    return z_init,eps0,T

# parts 2 and 5 of proj 2 (hopefully)

### part 2 :)

In [4]:
def pressure(rho, mue):
    """
    Arguments
        rho
            mass density (kg/m**3)
        mue
            baryon/electron ratio
    
    Returns
        electron degeneracy pressure (Pascal)
    """
    
    # replace following lines with body of routine
    m_u = 1.661e-27 # atomic mass unit in kg
    h = 6.62607015e-34 # Planck's constant -- m^2 * kg/s
    m_e = 9.1093837015e-31 # mass of an electron in kg
    p = (1/5) * ((3/(8*np.pi))**(2/3)) * ((h**2)/m_e) * ((rho/(mue * m_u))**(5/3)) # electron degeneracy pressure in Pa
    return p

### part 5 :)

In [1]:
def lengthscales(m,z,mue):
    """
    Computes the radial length scale H_r and the pressure length H_P
    
    Arguments
        m
            current mass coordinate (units = ?)
        z (array)
           [ r, p ] (units = ?)
        mue
            mean electron weight
    
    Returns
        z/|dzdm| (units = ?)
    """

    # fill this in
    # pass
    G = 6.6743e-11 # gravitational constant in m^3 kg^-1 s^-2
    radius = z[0]
    pressure = z[1]
    rho = density(pressure, mue)
    H_r = 4 * np.pi * (radius**3) * rho
    H_p = (4 * np.pi * (radius**4) * pressure) / (G * m) 
    return H_r, H_p

In [3]:
def integrate(Pc,delta_m,eta,xi,mue,max_steps=10000):
    """
    Integrates the scaled stellar structure equations

    Arguments
        Pc
            central pressure (units = ?)
        delta_m
            initial offset from center (units = ?)
        eta
            The integration stops when P < eta * Pc
        xi
            The stepsize is set to be xi*min(p/|dp/dm|, r/|dr/dm|)
        mue
            mean electron mass
        max_steps
            solver will quit and throw error if this more than max_steps are 
            required (default is 10000)
                        
    Returns
        m_step, r_step, p_step
            arrays containing mass coordinates, radii and pressures during 
            integration (what are the units?)
    """
        
    m_step = np.zeros(max_steps)
    r_step = np.zeros(max_steps)
    p_step = np.zeros(max_steps)
  
    # set starting conditions using central values
    z = central_values(Pc, delta_m, mue)
    some_fraction = 0.1
    Nsteps = 0
    for step in range(max_steps):
        radius = z[0]
        pressure = z[1]
        # are we at the surface?
        if (pressure < eta*Pc):
            break
        # store the step
        m_step[Nsteps] = m_step[Nsteps-1] + delta_m
        r_step[Nsteps] = radius
        p_step[Nsteps] = pressure
        
        # set the stepsize
        H_r, H_p = lengthscales(m_step[Nsteps], z, mue)
        h = some_fraction * min(H_r, H_p)

        # take a step
        
        
        # increment the counter
        Nsteps += 1
    # if the loop runs to max_steps, then signal an error
    else:
        raise Exception('too many iterations')
        
    return m_step[0:Nsteps],r_step[0:Nsteps],p_step[0:Nsteps]